In [1]:
# Re-define the possible options for each variable
goals = ["MAINTENANCE", "ENDURANCE", "PERFORMANCE"]
days = ["3", "4", "5", "6"]
levels = [str(i) for i in range(9)]  # "0" to "8"
injuries = ["LO-INJ", "MID-INJ", "HI-INJ"]
terrains = ["ROAD", "TRAIL"]

# Generate all possible combinations of the variables
filenames = [
    f"{day}-{goal}_{level}_{injury}_{terrain}"
    for day in days
    for goal in goals
    for level in levels
    for injury in injuries
    for terrain in terrains
]

# Export the combinations to a .txt file
with open("filename.txt", "w") as file:
    file.write("\n".join(filenames))

print("File 'filename.txt' has been created with all combinations.")

File 'filename.txt' has been created with all combinations.


In [8]:
import os
import random
import math
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter, landscape
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# Register the Epilogue font
pdfmetrics.registerFont(TTFont('Epilogue', 'Epilogue-VariableFont_wght.ttf'))

# Define colors from the provided theme
COLOR_THEME = {
    'background': "#FEFFE9",
    'highlight': "#FFFDC1",
    'primary': "#DFE777",
    'secondary': "#2E3105",
    'accent': "#B8BC7F"
}

def calculate_threshold_total_duration(level, week):
    k_threshold = 0.138
    full_duration = 20 * math.exp(k_threshold * level)
    # Adjust duration based on week (using 1.1^X progression)
    return full_duration / (1.1 ** (5 - week))

def calculate_threshold_workout_duration(level, week):
    threshold_total = calculate_threshold_total_duration(level, week)
    # Adjust workout duration based on week
    return 10 + (threshold_total - 20) * 0.67

def calculate_speed_total_duration(level, week):
    k_speed = 0.116
    full_duration = 20 * math.exp(k_speed * level)
    # Adjust duration based on week
    return full_duration / (1.1 ** (5 - week))

def calculate_speed_workout_duration(level, week):
    speed_total = calculate_speed_total_duration(level, week)
    # Adjust workout duration based on week
    return 10 + (speed_total - 20) * 0.5

def calculate_easy_duration(level, week):
    k_easy = 0.173
    full_duration = 15 * math.exp(k_easy * level)
    # Adjust duration based on week
    return full_duration / (1.1 ** (5 - week))

def calculate_long_duration(level, week):
    k_long = 0.173
    full_duration = 30 * math.exp(k_long * level)
    # Adjust duration based on week
    return full_duration / (1.1 ** (5 - week))

def round_to_nearest(value, multiple=2):
    """Rounds the value to the nearest multiple of the specified number (default is 2)."""
    return round(value / multiple) * multiple

def generate_simple_schedule(num_goal):
    """
    Generate a simple running schedule for a week based on the num_goal input.
    """
    templates = {
        "3-PERFORMANCE": ["Threshold", "Rest", "Threshold", "Rest", "Speed", "Rest", "Rest"],
        "4-PERFORMANCE": ["Threshold", "Rest", "Threshold", "Rest", "Speed", "Long Run", "Rest"],
        "5-PERFORMANCE": ["Threshold", "Easy", "Threshold", "Rest", "Speed", "Long Run", "Rest"],
        "6-PERFORMANCE": ["Threshold", "Easy", "Threshold", "Easy", "Speed", "Long Run", "Rest"],
        "3-ENDURANCE": ["Threshold", "Rest", "Threshold", "Rest", "Rest", "Long Run", "Rest"],
        "4-ENDURANCE": ["Threshold", "Rest", "Threshold", "Rest", "Easy", "Long Run", "Rest"],
        "5-ENDURANCE": ["Threshold", "Easy", "Threshold", "Easy", "Easy", "Long Run", "Rest"],
        "6-ENDURANCE": ["Threshold", "Easy", "Threshold", "Easy", "Easy", "Long Run", "Rest"],
        "3-MAINTENANCE": ["Easy", "Rest", "Threshold", "Rest", "Rest", "Long Run", "Rest"],
        "4-MAINTENANCE": ["Easy", "Rest", "Threshold", "Easy", "Long Run", "Rest"],
        "5-MAINTENANCE": ["Easy", "Easy", "Threshold", "Rest", "Easy", "Long Run", "Rest"],
        "6-MAINTENANCE": ["Easy", "Easy", "Threshold", "Easy", "Easy", "Long Run", "Rest"]
    }

    if num_goal not in templates:
        raise ValueError(f"Invalid goal: {num_goal}. Supported values are '3-PERFORMANCE', '4-PERFORMANCE', '5-PERFORMANCE', '6-PERFORMANCE', '3-ENDURANCE', '4-ENDURANCE', '5-ENDURANCE', '6-ENDURANCE', '3-MAINTENANCE', '4-MAINTENANCE', '5-MAINTENANCE', '6-MAINTENANCE'.")

    schedule = templates[num_goal]

    while len(schedule) < 7:
        schedule.append("Rest")

    schedule.append("Volume")

    return schedule

def generate_calendar_layout(filename):
    parts = filename.split('_')
    num_goal = parts[0]
    level = int(parts[1])
    injury_status = parts[2]  # LO-INJ, MID-INJ, or HI-INJ
    terrain = parts[3]

    workout_schedule = generate_simple_schedule(num_goal)

    # Injury-based modifications
    cross_training_replacements = 0  # Counter to track how many cross training replacements are needed
    
    if injury_status == "MID-INJ":
        cross_training_replacements = 1
    elif injury_status == "HI-INJ":
        cross_training_replacements = 2
    
    # First, check for Speed workouts
    if cross_training_replacements > 0:
        for i in range(len(workout_schedule)):
            if workout_schedule[i] == "Speed":
                workout_schedule[i] = "Cross Training"
                cross_training_replacements -= 1
                if cross_training_replacements == 0:
                    break  # Exit if we've replaced all required workouts
    
    # Next, check for Long Run workouts if replacements are still needed
    if cross_training_replacements > 0:
        for i in range(len(workout_schedule)):
            if workout_schedule[i] == "Long Run":
                workout_schedule[i] = "Cross Training"
                cross_training_replacements -= 1
                if cross_training_replacements == 0:
                    break  # Exit if we've replaced all required workouts
    
    # Finally, check for Threshold workouts if replacements are still needed
    if cross_training_replacements > 0:
        for i in range(len(workout_schedule)):
            if workout_schedule[i] == "Threshold":
                workout_schedule[i] = "Cross Training"
                cross_training_replacements -= 1
                if cross_training_replacements == 0:
                    break  # Exit if we've replaced all required workouts

    if not filename.endswith('.pdf'):
        filename += '.pdf'

    pdf_path = os.path.join(os.getcwd(), filename)
    c = canvas.Canvas(pdf_path, pagesize=landscape(letter))
    width, height = landscape(letter)

    margin = 50
    cell_width = (width - 2 * margin) / 8
    cell_height = 60  # Increased height for better text fit

    c.setFillColor(COLOR_THEME['background'])
    c.setStrokeColor(COLOR_THEME['secondary'])
    c.rect(0, 0, width, height, fill=True, stroke=True)

    day_labels = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday", "Volume"]
    y = height - margin - cell_height
    for i, day in enumerate(day_labels):
        x = margin + i * cell_width
        c.setFillColor(COLOR_THEME['highlight'] if i < 7 else COLOR_THEME['accent'])
        c.rect(x, y, cell_width, cell_height, fill=True, stroke=True)

        c.setFillColor(COLOR_THEME['secondary'])
        c.setFont("Epilogue", 14)
        text_width = c.stringWidth(day, "Epilogue", 14)
        c.drawString(x + (cell_width - text_width) / 2, y + cell_height / 2 - 7, day)

    y = height - margin - cell_height * 2

    for week in range(1, 6):  # Weeks are now 1 through 5
        total_volume = 0
        threshold_added = False
        for i in range(8):
            x = margin + i * cell_width
            volume_font_size = 8  # Set smaller font size for volume text
            if i < 7:
                workout = workout_schedule[i]
                if workout == "Rest":
                    c.setFillColor(COLOR_THEME['accent'])
                    volume_text = "Rest"
                elif workout == "Threshold":
                    total_duration = round_to_nearest(calculate_threshold_total_duration(level, week))
                    workout_duration = round_to_nearest(calculate_threshold_workout_duration(level, week))
                    if terrain == "TRAIL" and not threshold_added:
                        volume_text = f"Threshold*\nTotal: {total_duration} min\nWorkout: {workout_duration} min"
                        threshold_added = True  # Mark that the "*" has been added
                    else:
                        volume_text = f"Threshold\nTotal: {total_duration} min\nWorkout: {workout_duration} min"
                    c.setFillColor(COLOR_THEME['primary'])
                    total_volume += total_duration
                elif workout == "Speed":
                    total_duration = round_to_nearest(calculate_speed_total_duration(level, week))
                    workout_duration = round_to_nearest(calculate_speed_workout_duration(level, week))
                    volume_text = f"Speed\nTotal: {total_duration} min\nWorkout: {workout_duration} min"
                    c.setFillColor(COLOR_THEME['primary'])
                    total_volume += total_duration
                elif workout == "Easy":
                    total_duration = round_to_nearest(calculate_easy_duration(level, week))
                    volume_text = f"Easy\nTotal: {total_duration} min"
                    c.setFillColor(COLOR_THEME['highlight'])
                    total_volume += total_duration
                elif workout == "Long Run":
                    total_duration = round_to_nearest(calculate_long_duration(level, week))  # Adjust to cross-training duration
                    if terrain == "TRAIL":
                        volume_text = f"Long Run*\nTotal: {total_duration} min"
                    else:
                        volume_text = f"Long Run*\nTotal: {total_duration} min"
                    c.setFillColor(COLOR_THEME['primary'])
                    total_volume += total_duration
                elif workout == "Cross Training":
                    total_duration = round_to_nearest(calculate_long_duration(level, week))
                    volume_text = f"Cross\nTotal: {total_duration} min"
                    c.setFillColor(COLOR_THEME['primary'])
                    total_volume += total_duration
                else:
                    volume_text = "N/A"
            else:
                c.setFillColor(COLOR_THEME['highlight'])
                volume_text = f"{total_volume} min"
                volume_font_size = 12  # Use larger font for total volume
            
            # Draw the cell and the text inside it
            c.rect(x, y, cell_width, cell_height, fill=True, stroke=True)
            
            c.setFillColor(COLOR_THEME['secondary'])
            c.setFont("Epilogue", 14)  # Larger font size for the run type text
            lines = volume_text.split("\n")
            
            # Draw the main text (run type) - align it slightly below the top
            c.setFont("Epilogue", 14)
            for j, line in enumerate(lines[:1]):  # Only the first line (workout type)
                text_width = c.stringWidth(line, "Epilogue", 14)
                c.drawString(x + (cell_width - text_width) / 2, y + cell_height - 25, line)
            
            # Draw the volume text with smaller font size (4 points smaller)
            c.setFont("Epilogue", volume_font_size)
            for j, line in enumerate(lines[1:]):  # Start from the second line (volume text)
                text_width = c.stringWidth(line, "Epilogue", volume_font_size)
                c.drawString(x + (cell_width - text_width) / 2, y + cell_height - 25 - ((j + 1) * 15), line)
    
        y -= cell_height
    
    c.save()
    print(f"Calendar PDF saved to: {pdf_path}")

# Main script
file_path = 'filename.txt'

with open(file_path, 'r') as f:
    filenames = f.readlines()

filenames = [filename.strip() for filename in filenames]
random_filenames = random.sample(filenames, 5)

print(f"Randomly selected filenames: {random_filenames}")

for filename in random_filenames:
    print(f"Generating calendar for {filename}...")
    generate_calendar_layout(filename)

Randomly selected filenames: ['5-MAINTENANCE_3_MID-INJ_ROAD', '3-ENDURANCE_1_HI-INJ_ROAD', '4-ENDURANCE_4_HI-INJ_TRAIL', '5-PERFORMANCE_1_MID-INJ_ROAD', '6-PERFORMANCE_8_MID-INJ_ROAD']
Generating calendar for 5-MAINTENANCE_3_MID-INJ_ROAD...
Calendar PDF saved to: C:\Users\evane\OneDrive\Desktop\training_plan_generator\5-MAINTENANCE_3_MID-INJ_ROAD.pdf
Generating calendar for 3-ENDURANCE_1_HI-INJ_ROAD...
Calendar PDF saved to: C:\Users\evane\OneDrive\Desktop\training_plan_generator\3-ENDURANCE_1_HI-INJ_ROAD.pdf
Generating calendar for 4-ENDURANCE_4_HI-INJ_TRAIL...
Calendar PDF saved to: C:\Users\evane\OneDrive\Desktop\training_plan_generator\4-ENDURANCE_4_HI-INJ_TRAIL.pdf
Generating calendar for 5-PERFORMANCE_1_MID-INJ_ROAD...
Calendar PDF saved to: C:\Users\evane\OneDrive\Desktop\training_plan_generator\5-PERFORMANCE_1_MID-INJ_ROAD.pdf
Generating calendar for 6-PERFORMANCE_8_MID-INJ_ROAD...
Calendar PDF saved to: C:\Users\evane\OneDrive\Desktop\training_plan_generator\6-PERFORMANCE_8_MI